In [7]:
import time
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from utils.balance_data import Oversampler
from utils.base_models import BaseModels
from utils.numerical_scalers import NumericalScalers
from utils.categorical_encoders import CategoricalEncoders

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, ConfusionMatrixDisplay

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

### 1. Leer datasets

In [8]:
# 1. Leer el dataset
dataset = pd.read_parquet('./data/2_data_preprocesada.parquet')
dataset = dataset.drop(columns=['DF_TYPE'])

# Seleccionar aleatoriamente el 10% de los datos
dataset = dataset.sample(frac=0.01, random_state=42)
print(dataset.shape)
print(dataset['TARGET'].value_counts())

(13056, 14)
TARGET
0    6978
1    6078
Name: count, dtype: int64


### 2. Generar combinaciones de categóricos y numéricos

In [9]:
# 2. Separar TARGET, df_categorical y df_numeric
df_target = dataset['TARGET']
df_categorical = dataset.select_dtypes(include=['object'])
df_numeric = dataset.select_dtypes(include=['float64', 'int64'])

# 3. Eliminar TARGET de df_numeric
df_numeric = df_numeric.drop(columns=['TARGET'])

In [10]:
# 4. Generar las combinaciones de Encoder y Scaler
encoder_methods = ['LabelEncoder', 'OneHotEncoder', 'OrdinalEncoder', 'FrequencyEncoder', 'BinaryEncoder', 'BackwardDifferenceEncoder']
scaler_methods = ['StandardScaler', 'MinMaxScaler', 'MaxAbsScaler', 'RobustScaler', 'Normalizer', 'PowerTransformer']

# 5. Instanciar CategoricalEncoders
categorical = CategoricalEncoders(dataset=df_categorical)
binary_columns, categorical_columns = categorical.get_binary_categorical_columns()

# 6. Instanciar NumericalScalers
numerical = NumericalScalers(dataset=df_numeric)

# 7. Generar todas las combinaciones de Encoder y Scaler
def get_list_data_processed(encoder_methods, scaler_methods):
    combinations = list(itertools.product(encoder_methods, scaler_methods))
    
    i=0
    list_data_processed = []
    for encoder_method, scaler_method in combinations:
        data_encoded = categorical.provider(binary_columns, categorical_columns, method=encoder_method)
        data_scaled = numerical.provider(method=scaler_method)

        processed_data = pd.concat([data_encoded, data_scaled], axis=1)
        list_data_processed.append((f'{encoder_method} - {scaler_method}', processed_data))
        print(f'{str(i).zfill(2)} -> Encoder: {encoder_method} - Scaler: {scaler_method} - Shape: {processed_data.shape}')
        i+=1
    
    return list_data_processed

list_data_processed = get_list_data_processed(encoder_methods, scaler_methods)

00 -> Encoder: LabelEncoder - Scaler: StandardScaler - Shape: (13056, 13)
01 -> Encoder: LabelEncoder - Scaler: MinMaxScaler - Shape: (13056, 13)
02 -> Encoder: LabelEncoder - Scaler: MaxAbsScaler - Shape: (13056, 13)
03 -> Encoder: LabelEncoder - Scaler: RobustScaler - Shape: (13056, 13)
04 -> Encoder: LabelEncoder - Scaler: Normalizer - Shape: (13056, 13)
05 -> Encoder: LabelEncoder - Scaler: PowerTransformer - Shape: (13056, 13)
06 -> Encoder: OneHotEncoder - Scaler: StandardScaler - Shape: (13056, 38)
07 -> Encoder: OneHotEncoder - Scaler: MinMaxScaler - Shape: (13056, 38)
08 -> Encoder: OneHotEncoder - Scaler: MaxAbsScaler - Shape: (13056, 38)
09 -> Encoder: OneHotEncoder - Scaler: RobustScaler - Shape: (13056, 38)
10 -> Encoder: OneHotEncoder - Scaler: Normalizer - Shape: (13056, 38)
11 -> Encoder: OneHotEncoder - Scaler: PowerTransformer - Shape: (13056, 38)
12 -> Encoder: OrdinalEncoder - Scaler: StandardScaler - Shape: (13056, 13)
13 -> Encoder: OrdinalEncoder - Scaler: MinMax

### 3. Split test-train y balanceo de cada dataset

In [11]:
# 4. Generar los conjuntos de entrenamiento y prueba
def get_list_split_data(list_data_processed):
    i=0
    list_split_data = []
    for encoded_scaler, data_processed in list_data_processed:
        X = data_processed
        y = df_target
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        list_split_data.append((encoded_scaler, X_train, X_test, y_train, y_test))
        print(f'{str(i).zfill(2)} -> Split Encoded Scaler Method: {encoded_scaler}')
        i+=1

    return list_split_data

list_data_encoded_split = get_list_split_data(list_data_processed)

00 -> Split Encoded Scaler Method: LabelEncoder - StandardScaler
01 -> Split Encoded Scaler Method: LabelEncoder - MinMaxScaler
02 -> Split Encoded Scaler Method: LabelEncoder - MaxAbsScaler
03 -> Split Encoded Scaler Method: LabelEncoder - RobustScaler
04 -> Split Encoded Scaler Method: LabelEncoder - Normalizer
05 -> Split Encoded Scaler Method: LabelEncoder - PowerTransformer
06 -> Split Encoded Scaler Method: OneHotEncoder - StandardScaler
07 -> Split Encoded Scaler Method: OneHotEncoder - MinMaxScaler
08 -> Split Encoded Scaler Method: OneHotEncoder - MaxAbsScaler
09 -> Split Encoded Scaler Method: OneHotEncoder - RobustScaler
10 -> Split Encoded Scaler Method: OneHotEncoder - Normalizer
11 -> Split Encoded Scaler Method: OneHotEncoder - PowerTransformer
12 -> Split Encoded Scaler Method: OrdinalEncoder - StandardScaler
13 -> Split Encoded Scaler Method: OrdinalEncoder - MinMaxScaler
14 -> Split Encoded Scaler Method: OrdinalEncoder - MaxAbsScaler
15 -> Split Encoded Scaler Method

In [14]:
# 5. Aplicar balanceo de clases a cada list_data_encoded
oversampler = Oversampler()

def get_list_data_encoded_balanced(list_data_encoded_split):
    oversampler_methods = ['RandomOverSampler', 'SMOTE', 'ADASYN', 'BorderlineSMOTE', 'SVMSMOTE']
    
    i=0
    list_data_encoded_balanced = []
    for encoded_method, X_train, X_test, y_train, y_test in list_data_encoded_split:
        for oversampler_method in oversampler_methods:
            X_train_balanced, y_train_balanced = oversampler.provider(method=oversampler_method, X=X_train, y=y_train)
            
            list_data_encoded_balanced.append((encoded_method, 
                                               oversampler_method, 
                                               X_train_balanced, 
                                               X_test,
                                               y_train_balanced,
                                               y_test
                                               ))
            print(f'{str(i).zfill(2)}: Encoded method: {encoded_method} - Oversampler method: {oversampler_method} - Train shape: {X_train_balanced.shape} - Test shape: {X_test.shape}')
            i += 1

    return list_data_encoded_balanced

# Aplicar el balanceo a los datos de entrenamiento
list_data_encoded_split_balanced = get_list_data_encoded_balanced(list_data_encoded_split)

_, _, X_train, X_test, y_train, y_test = list_data_encoded_split_balanced[0]
print('\ny_train:', str(y_train.value_counts()))
print('\ny_test:', str(y_test.value_counts()))

00: Encoded method: LabelEncoder - StandardScaler - Oversampler method: RandomOverSampler - Train shape: (11164, 13) - Test shape: (2612, 13)
01: Encoded method: LabelEncoder - StandardScaler - Oversampler method: SMOTE - Train shape: (11164, 13) - Test shape: (2612, 13)


ValueError: Method should be 'RandomOverSampler', 'SMOTE', 'ADASYN', 'BorderlineSMOTE', 'SVMSMOTE', or 'KMeansSMOTE'

In [ ]:
# 2. Obtener las columnas binarias y categóricas
categorical = CategoricalEncoders(dataset=dataset)
binary_columns, categorical_columns = categorical.get_binary_categorical_columns()

# 3. Obtener los datos codificados
def get_list_data_encoded():
    encoded_methods = ['LabelEncoder', 'OneHotEncoder', 'OrdinalEncoder', 'FrequencyEncoder', 'BinaryEncoder', 'BackwardDifferenceEncoder']

    i = 0
    list_data_encoded = []
    for encoded_method in encoded_methods:
        data_encoded = categorical.provider(binary_columns, categorical_columns, method=encoded_method)
        list_data_encoded.append((encoded_method, data_encoded))
        print(f'{str(i).zfill(2)}: Encoded method: {encoded_method} - Data shape: {data_encoded.shape}')
        i += 1

    return list_data_encoded

list_data_encoded = get_list_data_encoded()

In [ ]:

def get_list_data_encoded_split(list_data_encoded, column_target_name, test_size=0.2, random_state=42):
    
    i=0
    list_data_split = []
    for method, data in list_data_encoded:
        X = data.drop(columns=[column_target_name])
        y = data[column_target_name]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        list_data_split.append((method, X_train, X_test, y_train, y_test))

        print(f'{str(i).zfill(2)}: Method: {method} - Train shape: {X_train.shape}, Test shape: {X_test.shape}')
        i += 1

    
    return list_data_split

# Uso de la función
list_data_encoded_split = get_list_data_encoded_split(list_data_encoded, column_target_name='Target')

_, X_train, X_test, y_train, y_test = list_data_encoded_split[0]
print('\ny_train:', str(y_train.value_counts()))
print('\ny_test:', str(y_test.value_counts()))

In [ ]:
# 4. Aplicar balanceo de clases a cada list_data_encoded
oversampler = Oversampler()

def get_list_data_encoded_balanced(list_data_encoded_split):
    oversampler_methods = ['random', 'smote', 'adasyn', 'borderlinesmote', 'svmsmote']
    
    i=0
    list_data_encoded_balanced = []
    for encoded_method, X_train, X_test, y_train, y_test in list_data_encoded_split:
        for oversampler_method in oversampler_methods:
            X_train_balanced, y_train_balanced = oversampler.provider(method=oversampler_method, X=X_train, y=y_train)
            
            list_data_encoded_balanced.append((encoded_method, 
                                               oversampler_method, 
                                               X_train_balanced, 
                                               X_test,
                                               y_train_balanced,
                                               y_test
                                               ))
            print(f'{str(i).zfill(2)}: Encoded method: {encoded_method} - Oversampler method: {oversampler_method} - Train shape: {X_train_balanced.shape} - Test shape: {X_test.shape}')
            i += 1

    return list_data_encoded_balanced

# Aplicar el balanceo a los datos de entrenamiento
list_data_encoded_split_balanced = get_list_data_encoded_balanced(list_data_encoded_split)

_, _, X_train, X_test, y_train, y_test = list_data_encoded_split_balanced[0]
print('\ny_train:', str(y_train.value_counts()))
print('\ny_test:', str(y_test.value_counts()))

In [ ]:
from sklearn.metrics import roc_auc_score
from utils.base_models import BaseModels

# Inicializar BaseModels y definir los nombres de los modelos
base_models = BaseModels()
name_models = ['logistic_regression', 'decision_tree', 'random_forest',
               'gradient_boosting', 'svm', 'knn', 'naive_bayes', 'mlp',
               'lgbm', 'catboost', 'xgboost']

i = 0
all_results = []
for encoded_method, oversampler_method, X_train, X_test, y_train, y_test in list_data_encoded_split_balanced:
    results = []
    for name in name_models:
        model = base_models.provider(name)
        model.fit(X_train, y_train)
        
        predict_train = model.predict_proba(X_train)[:, 1]
        predict_test = model.predict_proba(X_test)[:, 1]

        train_auc = roc_auc_score(y_train, predict_train)
        test_auc = roc_auc_score(y_test, predict_test)

        results.append((name, train_auc, test_auc))

        print(f"{str(i).zfill(2)}: Encoded Method: {encoded_method} - Oversampler Method: {oversampler_method} - AUC on training data with {name}: {train_auc:.3f}")
        print(f"AUC on testing data with {name}: {test_auc:.3f}")
        i += 1

    # Ordenar los resultados por test_auc de mayor a menor
    results.sort(key=lambda x: x[2], reverse=True)

    # Desempaquetar los resultados ordenados y almacenar con el método de codificación y balanceo
    sorted_names, train_aucs, test_aucs = zip(*results)
    all_results.append((encoded_method, oversampler_method, sorted_names, train_aucs, test_aucs))


In [ ]:
import matplotlib.pyplot as plt

# Consolidar todos los resultados en una lista
consolidated_results = []
for encoded_method, oversampler_method, sorted_names, train_aucs, test_aucs in all_results:
    for name, train_auc, test_auc in zip(sorted_names, train_aucs, test_aucs):
        full_model_name = f"{encoded_method} - {oversampler_method} - {name}"
        consolidated_results.append((full_model_name, train_auc, test_auc))

# Ordenar los resultados por test_auc de mayor a menor y seleccionar el top 10
consolidated_results.sort(key=lambda x: x[2], reverse=False)
top_results = consolidated_results[:15]

# Desempaquetar los resultados del top 10
top_names, top_train_aucs, top_10_test_aucs = zip(*top_results)

plt.figure(figsize=(14, 9))

# Gráfico de líneas para el top 10
plt.plot(top_names, top_train_aucs, label='Training AUC', marker='o', color='skyblue')
plt.plot(top_names, top_10_test_aucs, label='Testing AUC', marker='o', color='salmon')

# Añadir los valores a los puntos
for i, txt in enumerate(top_train_aucs):
    plt.annotate(f'{txt:.3f}', (top_names[i], top_train_aucs[i]), textcoords="offset points", xytext=(0,10), ha='center', color='blue')
for i, txt in enumerate(top_10_test_aucs):
    plt.annotate(f'{txt:.3f}', (top_names[i], top_10_test_aucs[i]), textcoords="offset points", xytext=(0,-15), ha='center', color='red')

plt.xlabel('Modelos')
plt.ylabel('AUC')
plt.title('Top of 330 Models by Testing AUC')
plt.xticks(rotation=90, fontsize=9)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Consolidar todos los resultados en una lista
consolidated_results = []
for encoded_method, oversampler_method, sorted_names, train_aucs, test_aucs in all_results:
    for name, train_auc, test_auc in zip(sorted_names, train_aucs, test_aucs):
        full_model_name = f"{encoded_method} - {oversampler_method} - {name}"
        consolidated_results.append((full_model_name, train_auc, test_auc))

# Ordenar los resultados por test_auc de mayor a menor y seleccionar el top 10
consolidated_results.sort(key=lambda x: x[2], reverse=False)
top_results = consolidated_results[-15:]

# Desempaquetar los resultados del top 10
top_names, top_train_aucs, top_10_test_aucs = zip(*top_results)

plt.figure(figsize=(14, 9))

# Gráfico de líneas para el top 10
plt.plot(top_names, top_train_aucs, label='Training AUC', marker='o', color='skyblue')
plt.plot(top_names, top_10_test_aucs, label='Testing AUC', marker='o', color='salmon')

# Añadir los valores a los puntos
for i, txt in enumerate(top_train_aucs):
    plt.annotate(f'{txt:.3f}', (top_names[i], top_train_aucs[i]), textcoords="offset points", xytext=(0,10), ha='center', color='blue')
for i, txt in enumerate(top_10_test_aucs):
    plt.annotate(f'{txt:.3f}', (top_names[i], top_10_test_aucs[i]), textcoords="offset points", xytext=(0,-15), ha='center', color='red')

plt.xlabel('Modelos')
plt.ylabel('AUC')
plt.title('Top of 330 Models by Testing AUC')
plt.xticks(rotation=90, fontsize=9)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
